# Moderate content using Amazon Rekognition

***
This notebook provides a walkthrough of [content moderation APIs](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) in Amazon Rekognition. You can quickly identify inappropriate content in your video and image libraries.
***

# Initialize stuff

In [1]:
# Initialise Notebook
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [2]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

In [3]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [4]:
# S3 bucket that contains sample images and videos

# We are providing sample images and videos in this bucket so
# you do not have to manually download/upload test images and videos.

bucketName = "aws-workshops-" + awsRegion

In [5]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and drwaw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir m1tmp
tempFolder = 'm1tmp/'

mkdir: cannot create directory ‘m1tmp’: File exists


# Content moderation in Images
***

In [6]:
imageName = "object-detection/bikini.png"

#### Call Rekognition to detect objects in the image

In [7]:
# Call Amazon Rekognition to detect unsafe content in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html

detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [8]:
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucketName, 'Key': imageName})))

#### Review the raw JSON reponse from Rekognition

In [9]:
# Show JSON response returned by Rekognition Moderation API
# In the JSON response below, you will see Moderation Labels, confidence score and additional information.

display(detectModerationLabelsResponse)

{'ModerationLabels': [{'Confidence': 98.7787857055664,
   'Name': 'Female Swimwear Or Underwear',
   'ParentName': 'Suggestive'},
  {'Confidence': 98.7787857055664, 'Name': 'Suggestive', 'ParentName': ''}],
 'ModerationModelVersion': '3.0',
 'ResponseMetadata': {'RequestId': '6339d8f6-1e8c-4393-ad2f-07c3715ef023',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 14 Oct 2019 22:44:00 GMT',
   'x-amzn-requestid': '6339d8f6-1e8c-4393-ad2f-07c3715ef023',
   'content-length': '217',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

#### Display list of detected moderation labels

In [10]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

- Female Swimwear Or Underwear (Confidence: 98.7787857055664)
  - Parent: Suggestive
- Suggestive (Confidence: 98.7787857055664)
  - Parent: 


# Content moderation in videos

Content Moderation in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
 We first start content moderation job which returns a Job Id.
 We can then call get_content_moderation to get the job status and after job is complete, we can get moderation results.
 In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
***

In [11]:
videoName = "celebrity-rekognition/media/GrandTour720.mp4"

#### Call Rekognition to start a job for content moderation

In [12]:
# Start celebrity recognition job
startModerationLabelDetection = rekognition.start_content_moderation(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

'Job Id: d2bfe64a789c8e0da33d1a8933004e78076b153ce9b7f973679291f0bc73f3c3'

### Additional (Optional) Request Attributes

ClientRequestTokenL
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-ClientRequestToken

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-JobTag

MinConfidence:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-MinConfidence

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html#rekognition-StartContentModeration-request-NotificationChannel


#### Wait for content moderation job to complete

In [ ]:
# Wait for object detection job to complete
# In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')
    
display(getContentModeration['JobStatus'])

...........

#### Review raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Object Detection API
# In the JSON response below, you will see list of detected objects and activities.
# For each detected object, you will see information like Timestamp

display(getContentModeration)

#### Dislpay names of recognized celebrities in the video

In [ ]:
theObjects = {}

# Display timestamps and celebrites detected at that time
strDetail = "Moderation labels in video<br>=======================================<br>"
strOverall = "Moderation labels in the overall video:<br>=======================================<br>"

# Celebrities detected in each frame
for obj in getContentModeration['ModerationLabels']:
    ts = obj ["Timestamp"]
    cconfidence = obj['ModerationLabel']["Confidence"]
    oname = obj['ModerationLabel']["Name"]
    strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, oname, round(cconfidence,2))
    if oname in theObjects:
        cojb = theObjects[oname]
        theObjects[oname] = {"Name" : oname, "Count": 1+cojb["Count"]}
    else:
        theObjects[oname] = {"Name" : oname, "Count": 1}

# Unique objects detected in video
for theObject in theObjects:
    strOverall = strOverall + "Name: {}, Count: {}<br>".format(theObject, theObjects[theObject]["Count"])

# Display results
display(HTML(strOverall))
#display(HTML(strDetail))

#### Show video in the player

In [ ]:
# Show video in a player

s3FilePrefix = "https://s3.amazonaws.com"
if(not awsRegion == 'us-east-1'):
    s3FilePrefix = "https://s3-{}.amazonaws.com".format(awsRegion)

s3VideoUrl = "{0}/{1}/{2}".format(s3FilePrefix, bucketName, videoName)

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3VideoUrl)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(videoTag)

display(HTML(videoui))

In [ ]:
listui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(strDetail)
display(HTML(listui))

## Comprehensive content moderation
***
Now let us try an image where we will use three different APIs of Amazon Textract to detected inappropriate content, objects of interest and inappropriate text.

In [ ]:
imageName = "object-detection/bikini-text.png"

In [ ]:
psUrl = s3.generate_presigned_url('get_object', Params={'Bucket': bucketName,
                                                            'Key': imageName})

In [ ]:
display(HTML("<img src='{}'/>".format(psUrl)))

In [ ]:
# Call Amazon Rekognition to detect objects in the image
detectLabelsResponse = rekognition.detect_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
detectTextResponse = rekognition.detect_text(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
foundUnsafeObject = False

unsafeObjects = ["Bikini", "Gun"]
for label in detectLabelsResponse["Labels"]:
    if(label["Name"] in unsafeObjects):
        print("Found unsafe object: {}".format(label["Name"]))
        foundUnsafeObject = True

In [ ]:
foundUnsafeLabel = False
unsafeModerationLabels = ["Suggestive", "Explicit"]
for mlabel in detectModerationLabelsResponse["ModerationLabels"]:
    if(mlabel["ParentName"] in unsafeModerationLabels):
        print("Found unsafe content {} under category {}".format(mlabel["Name"], mlabel["ParentName"]))
        foundUnsafeLabel = True

In [ ]:
foundUnsafeText = False
unsafeText = ["crap", "sh**"]
for td in detectTextResponse["TextDetections"]:
    if(td["Type"] == "WORD" and td["DetectedText"].lower() in unsafeText):
        print("Found unsafe text: {}".format(td["DetectedText"]))
        foundUnsafeText = True

In [ ]:
if(foundUnsafeObject or foundUnsafeLabel or foundUnsafeText):
    print("Unsafe content detected.")

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectModerationLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetContentModeration.html

***